<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

**Задание 1.**

***Обязательная часть***

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

***Дополнительная часть (необязательная)***

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>



**Решение задания 1**

***(сразу в формате доп части)***

In [11]:
url = 'https://habr.com/ru/search/page'



def get_all_links(url, query, page):
    """
    Функция собирает ссылки по страницам поисковой выдачи на сайте habr.com 
    
    """
    all_refs = []
    params = {
        'q': query
    }
# На этом сайте конкретного параметра, указывающего на страницу поисковой выдачи нет (ну,или я не нашел), но я заметил,
# со второй страницы в дресной строкепосле /search/  появляется /page. Поэксперементировав я понял, что несмотря на отсутствие
# page в адресной строке на первой странице поисковой выдачи, при ее добавлении все также работает.
# Дальше я просто создал цикл  с добавлением к адресу индекса страницы
    for i in range(1, page): # для индекса i в диапазоне от 1 до значения в переменной page
        url1 = url + str(i) # в котором мы создаем переменную с новым адресом, добавляя str(индекс i) в конце
        req = requests.get(url1, params) # делаем запрос,используя новый адрес и параметры
        soup = BeautifulSoup(req.text, 'html.parser') # читаем страницу в суп
        headers = soup.find_all('h2', class_='post__title') # собираем заголовки с ссылками на отдельные страницы статей
        r_list = list(map(lambda x: x.find('a').get('href'), headers)) # собираем в список ссылки на статьи
        all_refs += list(map(lambda x: x, r_list)) # собираем все списки по каждой странице поисковой выдачи в один большой список
    return all_refs # возвращаем один большой список


# в переменной page специально поставил 3 (хотя, там 35 страниц по требуемому поисковому запросу вылезает)
# поскольку комп слишком долго соображает, формируя итоговый Data frame
habr_refs = get_all_links(url, 'python, парсинг', 3)
    
len(habr_refs)

40

In [6]:
all_rows = pd.DataFrame() # заводим data frame
# Далее в цикле по каждой ссылке из списка habr_refs:
for hub in habr_refs: 
    pars = BeautifulSoup(requests.get(hub).text, 'html.parser') #создаем soup
    date = pd.to_datetime(pars.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date() #находим дату и переводим ее в нужный формат (без времени)
    title = pars.find('span', class_='post__title-text').text #находим заголовок и транслируем его в переменную title в текстовом формате
    row = {'date': date, 'title': title, 'link': hub} # создаем словарь, где ключами будут будущие названия столбцов,  а значениями дата, заголовок и ссылка (hub)
    all_rows = pd.concat([all_rows, pd.DataFrame([row])]) # с помощью конкатенации загоняем словарь в пустой DataFrame
all_rows.reset_index().drop('index', 1) #Сбрасываем индекс и удаляем лишний стобец с 0 индексами

,date,title,link
0,2011-06-14,Парсинг на Pуthon. Как собрать архив Голубятен,https://habr.com/ru/post/121815/
1,2020-10-01,Как выбрать решение для парсинга сайтов: класс...,https://habr.com/ru/post/521646/
2,2020-07-31,Как проанализировать рынок фотостудий с помощь...,https://habr.com/ru/post/512208/
3,2015-03-06,Веб-парсинг на Ruby,https://habr.com/ru/post/252379/
4,2014-10-28,Вглубь Pyparsing: парсим единицы измерения на ...,https://habr.com/ru/post/241670/
5,2019-05-07,Парсинг сайтов — а это вообще легально в России?,https://habr.com/ru/post/450834/
6,2020-03-21,Реализуем преобразования кода на Python,https://habr.com/ru/company/piter/blog/493424/
7,2011-09-02,Grab — python библиотека для парсинга сайтов,https://habr.com/ru/post/127584/
8,2013-12-18,Python на Хабре,https://habr.com/ru/post/205944/
9,2016-03-03,Теория и практика парсинга исходников с помощь...,https://habr.com/ru/company/pt/blog/210772/


**Задание 2.**

***Обязательная часть***

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

***Дополнительная часть (необязательная)***

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

**Решение задачи 2 (Обязательная часть)**

Было неинтересно искать информацию по почтовым ящикам [xxx@x.ru, yyy@y.com] и решил проверить свои ящики, в числе которых один актуальный, а два покрытые мхом и пылью почтовых серверов mail.ru. 

***По поводу дополнительной части***

Начал изучать документацию пополучению токена (вот тут: https://vk.com/dev/implicit_flow_user)
в результате собрал следующую строку запроса:
https://oauth.vk.com/authorize?client_id=17299154&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=groups&response_type=token&state=yow_man

но на него прилетает следующая ошибка:

    {"error":"invalid_request","error_description":"application was deleted"}

Написал вопрос
в slack по этой ошибке, но там тишина......

In [8]:
# строка запроса в переменную avast_url
avast_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
avast_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.19 Safari/537.36 OPR/76.0.4009.0 (Edition developer)',
    'Vaar-Version':'0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version':'1.0.0'
    }
# искомые email:
emails = {'emailAddresses': ["v.tumeneff@gmail.com", "tumeneff@bk.ru", "tumengoblin@mail.ru"]}


In [9]:
avast_resp = requests.post(avast_url, json = emails, headers = avast_headers) # запрос с указанием всего необходимого
avast_out = avast_resp.json() # заносим полученный ответ в формате json в переменную avast_out (получая тем самым словарь)

leaks_out = pd.DataFrame()

# Вычленяем из avast_out кусок с ключом 'summary' и упрощаем его, убирая лишние вложения.
summary = {k: list(v.values())[0] for (k,v) in avast_out['summary'].items()} # суровый dict comprehention

# в цикле сначала определяем, откуда берем почту, потом спускаемся до значений по ключу 'breaches', которые сами по себе 
# являются отдельным словарем, и проходимся по его ключам и значениям
for mail, ids in summary.items():
    mail_info = mail
    for breach, data in avast_out['breaches'].items(): # в значении по ключу 'breaches', (которое само по себе есть словарь):
        if int(breach) in ids: # если int(ключ) в этом подсловаре есть в знчениях словаря summary: 
            date_info = data['publishDate'] # в переменную date_info заносим значение по ключу publishDate
            descript_info = data['description']  # в переменную descript_info заносим значение по ключу description
            site_info = data['site'] # в переменную site_info заносим значение по ключу site
            row_avast = {'email': mail_info, 'date':date_info, 'site': site_info, 'description': descript_info} # все в словарь
            leaks_out = pd.concat([leaks_out, pd.DataFrame([row_avast])])  # методом конкатенации словари по каждой строке запихиваем в data frame leaks_out

# Наводим красоту: применяя метод .mask убраем повторяющиеся почтовые ящики
leaks_out['email']=leaks_out.email.mask(leaks_out.email.duplicated(),'')
leaks_out.reset_index().drop('index', 1) #Сбрасываем индекс и удаляем лишний столбец 'index'

,email,date,site,description
0,v.tumeneff@gmail.com,2016-12-15T00:00:00Z,vedomosti.ru,"In May 2012, Russian business newspaper Vedomo..."
1,tumeneff@bk.ru,2018-10-04T00:00:00Z,autowebinar.fm,"In December 2013, AutoWebinar FM's database wa..."
2,,2017-06-14T00:00:00Z,nnm-club.me,"In September 2013, Russian torrent tracker NNM..."
3,tumengoblin@mail.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
4,,2020-05-07T00:00:00Z,procapital.ru,"At an unconfirmed date, the Russian investment..."
5,,2020-12-03T00:00:00Z,egrul-piter.ru,"In November 2020, a collection of over 23,000 ..."
6,,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
7,,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
